In [1]:
import QuantLib as ql

In [2]:
calendar = ql.TARGET()

In [3]:
settlementDate = ql.Date(22, ql.September, 2004)

In [4]:
settlementDate = calendar.adjust(settlementDate)

In [5]:
fixingDays = 2

In [6]:
todaysDate = calendar.advance(settlementDate, -fixingDays, ql.Days)

In [7]:
ql.Settings.instance().evaluationDate = todaysDate

In [8]:
d1wQuote=0.0382

In [9]:
fra3x6Quote=0.037125

In [10]:
fut1Quote=96.2875

In [11]:
s2yQuote=0.037125

In [12]:
d1wRate = ql.SimpleQuote(d1wQuote)

In [13]:
fra3x6Rate = ql.SimpleQuote(fra3x6Quote)

In [14]:
fut1Price = ql.SimpleQuote(fut1Quote)

In [15]:
s2yRate = ql.SimpleQuote(s2yQuote)

In [16]:
depositDayCounter = ql.Actual360()

In [17]:
d1w = ql.DepositRateHelper(
            ql.QuoteHandle(d1wRate),
            ql.Period("1W"), fixingDays,
            calendar, ql.ModifiedFollowing,
            True, depositDayCounter)

In [18]:
fra3x6 = ql.FraRateHelper(
            ql.QuoteHandle(fra3x6Rate),
            3, 6, fixingDays, calendar, ql.ModifiedFollowing,
            True, depositDayCounter)

In [19]:
futMonths = 3

In [20]:
imm = ql.IMM.nextDate(settlementDate)

In [21]:
fut1 = ql.FuturesRateHelper(
            ql.QuoteHandle(fut1Price),
            imm,
            futMonths, calendar, ql.ModifiedFollowing,
            True, depositDayCounter, ql.QuoteHandle())

In [22]:
swFixedLegFrequency = ql.Annual

In [23]:
swFixedLegConvention = ql.Unadjusted

In [24]:
swFixedLegDayCounter = ql.Thirty360(ql.Thirty360.European)

In [25]:
swFloatingLegIndex = ql.Euribor6M()

In [26]:
s2y = ql.SwapRateHelper(
            ql.QuoteHandle(s2yRate), ql.Period("2Y"),
            calendar, swFixedLegFrequency,
            swFixedLegConvention, swFixedLegDayCounter,
            swFloatingLegIndex)

In [27]:
termStructureDayCounter =ql.ActualActual(ql.ActualActual.ISDA)

In [28]:
tolerance = 1.0e-15

In [29]:
depoSwapTermStructure = ql.PiecewiseLogLinearDiscount(settlementDate, [d1w, s2y],
                                          termStructureDayCounter)

In [30]:
depoFutSwapTermStructure = ql.PiecewiseLogLinearDiscount(settlementDate, [d1w, fut1, s2y],
                                          termStructureDayCounter)

In [31]:
depoFRASwapTermStructure = ql.PiecewiseLogLinearDiscount(settlementDate, [d1w, fra3x6, s2y],
                                          termStructureDayCounter)

In [32]:
discountingTermStructure = ql.RelinkableYieldTermStructureHandle()

In [33]:
forecastingTermStructure = ql.RelinkableYieldTermStructureHandle()

In [34]:
forecastingTermStructure.linkTo(depoSwapTermStructure)

In [35]:
discountingTermStructure.linkTo(depoSwapTermStructure)

In [36]:
swap = s2y.swap()

In [37]:
engine = ql.DiscountingSwapEngine(discountingTermStructure)

In [38]:
swap.setPricingEngine(engine)

In [39]:
swap.NPV()

0.14062095799386995